In [ ]:
import flopy
import os
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geopandas as gpd

In [ ]:

m = basic.load_model(verbose = True)

In [ ]:
m.plot()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = m.modelgrid.get_xcellcenters_for_layer(1)
y = m.modelgrid.get_ycellcenters_for_layer(1)

In [ ]:
# print layer bottom averages
# plot cell thicknesses
for lay in range(m.dis.nlay):
    
    ar = m.dis.get_cell_volumes()[lay]/(200*200)
    ar[m.bas6.ibound.array[lay]==0]=np.nan
    
    #print model botm
    bot = m.dis.getbotm()[lay]
    bot[m.bas6.ibound.array[lay]==0]=np.nan
    print("layer {}, elevation {:.1f}".format(lay,np.nanmean(bot.reshape((1,-1)))))
    


In [ ]:
swr = gpd.read_file("GIS/nhd_hr_demo_sfr_cells.shp")
swr = swr.query("name=='Russian River'")
swr

In [ ]:
swr_active = gpd.read_file("GIS/SWR_Reaches.shp")

ar = m.dis.gettop()
ar[m.bas6.ibound.array[lay]==0]=np.nan
swr_active.loc[:,'modtop'] = ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] 

ar = m.dis.botm.array[0]
# ar[m.bas6.ibound.array[lay]==0]=np.nan
swr_active.loc[:,'bot'] = ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] 

ar = m.dis.botm.array[1]
# ar[m.bas6.ibound.array[lay]==0]=np.nan
swr_active.loc[:,'lay2'] = ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] 

ar = m.dis.botm.array[2]
# ar[m.bas6.ibound.array[lay]==0]=np.nan
swr_active.loc[:,'lay3'] = ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] 

# swr_active.lo
fig,ax = plt.subplots(figsize=(10,4))

ax.scatter(swr_active.loc[:,'rno'],swr_active.loc[:,'modtop'], label = 'modtop',marker= '.', c = 'k')
ax.scatter(swr_active.loc[:,'rno'],swr_active.loc[:,'bot'], label = 'layer 1 bottom',marker= '.',  c = 'orange' )
ax.scatter(swr_active.loc[:,'rno'],swr_active.loc[:,'thalweg'], label = 'thalweg' )
ax.scatter(swr_active.loc[:,'rno'],swr_active.loc[:,'lay2'], label = 'layer 2 bottom',marker= '.',  c = 'k' )
ax.scatter(swr_active.loc[:,'rno'],swr_active.loc[:,'lay3'], label = 'layer 3 bottom',marker= '.',  c = 'k' )
ax.legend()
ax.set_title('Layer elevations and Thalweg for SWR Reaches')

In [ ]:
import warnings
# plot cell thicknesses
lay = 0

ar = m.dis.gettop()
ar[m.bas6.ibound.array[lay]==0]=np.nan

#burn in swr thalwegs

ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] = swr_active.loc[:,'thalweg']

fig, ax = plt.subplots(figsize = (10,10))
mapview = flopy.plot.PlotMapView(m,ax = ax)
linecollection = mapview.plot_grid(linewidth = .3)

quadmesh = mapview.plot_array(ar, vmax = 100,vmin  =0,cmap = 'gist_ncar_r')
ctx.add_basemap(ax, crs = 2226)
ax.set_title(f"layer {lay+1} elevation (feet)")

zeros = np.where(ar<0)

swr.plot(ax =ax, facecolor = 'None', edgecolor = 'k')

#check if any 
if zeros[0].size >0:
    warnings.warn('zero thickness layer found')
    xx , yy = x[0,zeros[1]], y[zeros[0],0]
    pt = gpd.GeoDataFrame(np.arange(int(len(zeros)/2)),geometry = gpd.points_from_xy(xx,yy), crs = 2226)
    pt.plot(ax = ax, markersize = 10, zorder = 10, facecolor = 'red')

cb = plt.colorbar(quadmesh, shrink=0.5)
plt.savefig(os.path.join('model summary', f"lay 1 top.png"))

In [ ]:
import warnings
# plot cell thicknesses
lay = 0

ar = m.dis.gettop()
ar[m.bas6.ibound.array[lay]==0]=np.nan

#burn in swr thalwegs

ar[swr_active.loc[:,'i'], swr_active.loc[:,'j']] = swr_active.loc[:,'thalweg']

fig, ax = plt.subplots(figsize = (10,10))
mapview = flopy.plot.PlotMapView(m,ax = ax)
linecollection = mapview.plot_grid(linewidth = .3)

quadmesh = mapview.plot_array(ar, vmax = 100,vmin  =0,cmap = 'gist_ncar_r')
ctx.add_basemap(ax, crs = 2226)
ax.set_title(f"layer {lay+1} elevation (feet)")

zeros = np.where(ar<0)

swr.plot(ax =ax, facecolor = 'None', edgecolor = 'k')

#check if any 
if zeros[0].size >0:
    warnings.warn('zero thickness layer found')
    xx , yy = x[0,zeros[1]], y[zeros[0],0]
    pt = gpd.GeoDataFrame(np.arange(int(len(zeros)/2)),geometry = gpd.points_from_xy(xx,yy), crs = 2226)
    pt.plot(ax = ax, markersize = 10, zorder = 10, facecolor = 'red')

cb = plt.colorbar(quadmesh, shrink=0.5)
plt.savefig(os.path.join('model summary', f"lay 1 top.png"))

In [ ]:
# plot cell thicknesses
for lay in range(m.dis.nlay):
    
    ar = m.dis.get_cell_volumes()[lay]/(200*200)
    ar[m.bas6.ibound.array[lay]==0]=np.nan
    

    fig, ax = plt.subplots(figsize = (10,10))
    mapview = flopy.plot.PlotMapView(m,ax = ax)
    linecollection = mapview.plot_grid(linewidth = .3)
    quadmesh = mapview.plot_array(ar, vmax = 100,vmin  =0,cmap = 'Blues')
    ctx.add_basemap(ax, crs = 2226)
    ax.set_title(f"layer {lay+1} thickness (feet)")
    
    zeros = np.where(ar<0)
    
    if zeros[0].size >0:
        xx , yy = x[0,zeros[1]], y[zeros[0],0]
        pt = gpd.GeoDataFrame(np.arange(int(len(zeros)/2)),geometry = gpd.points_from_xy(xx,yy), crs = 2226)
        pt.plot(ax = ax, markersize = 10, zorder = 10, facecolor = 'red')
        
    swr.plot(ax =ax, facecolor = 'None', edgecolor = 'k')
    
    cb = plt.colorbar(quadmesh, shrink=0.5)
    plt.savefig(os.path.join('model summary', f"lay thick {lay+1}.png"))
    # plt.imshow(m.dis.get_cell_volumes()[0])

In [ ]:
riv = flopy.modflow.ModflowRiv.load('RR_2022/Results/SWRSample05.riv', m, nper = 1)

In [ ]:
riv.stress_period_data[1]

In [ ]:
riv.export('GIS/riv_erase.shp',kper = 1)

In [ ]:
riv_gis = gpd.read_file('GIS/riv_erase.shp')


In [ ]:
for i in range(5):
    print(i)

In [ ]:
reaches.loc[:,'ROW']

In [ ]:
reaches.loc[:,'COL']

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:
reaches

In [ ]:
m.dis.top.export('GIS/mod_top.shp', )

In [ ]:

    
    
def add_top(df, m):
    modelgrid = m.modelgrid
    df.loc[:,'modeltop'] = modelgrid.top[df.ROW.values.astype(int)-1,df.COL.values.astype(int)-1]
    
    # df.loc[:,'geometry'] = modelgrid.get_s
    return df
    
    



reaches = get_swr_reaches()

reaches = add_top(reaches,m)

reaches.to_file('GIS/SWR_reaches.shp')

fig, ax = plt.subplots(figsize = (10,10))
mapview = flopy.plot.PlotMapView(m,ax = ax)
mapview.plot_ibound()
linecollection = mapview.plot_grid(linewidth = .3)
ctx.add_basemap(ax, crs = 2226)
reaches.plot('modeltop',ax = ax)
reaches

In [ ]:

n = 10
fig = plt.figure(figsize=(8, n*2))
for cnt, row in enumerate (np.linspace(0, m.dis.nrow-1, n, dtype = int)):
    ax = fig.add_subplot(n, 1, cnt+1)
    
    extend = (0, 26400, -100, 200)
    
    xsect = flopy.plot.PlotCrossSection(model=m, line={"Row": row},
                                       extent = extend)
    patches = xsect.plot_ibound()
    linecollection = xsect.plot_grid()
    # t = ax.set_title("Row {} Cross-Section with IBOUND Boundary Conditions".format(row))
    ax.text(0,1,"Row {}".format(row),
           transform = ax.transAxes, va = 'top')
    
    plt.savefig(os.path.join('model summary','xsections.png'))


In [ ]:
# def plot_riv_points():
#     riv = flopy.modflow.ModflowRiv.load('RR_2022/Results/SWRSample05.riv', m)
#     riv_points = gpd.GeoDataFrame(np.arange(riv.stress_period_data.data[0].k.shape[0]),
#        geometry = gpd.points_from_xy( x[0][riv.stress_period_data.data[0].j], 
#                                      y[:,0][riv.stress_period_data.data[0].i]), crs = 2226)

#     fig, ax = plt.subplots(figsize = (10,10))
#     mapview = flopy.plot.PlotMapView(m,ax = ax)
#     linecollection = mapview.plot_grid(linewidth = .3)

#     ctx.add_basemap(ax, crs = 2226)

#     # riv_points.plot(ax=ax)
    
#     riv_gis = gpd.read_file('GIS/riv_erase.shp')
#     riv_gis 
#     riv_gis.head().plot(ax = ax)
    
# plot_riv_points()

In [ ]:
riv.plot(kper = 1,ibound = False)

In [ ]:
flopy.modflow.ModflowBcf(ml)

In [ ]:
mnew = flopy.modflow.Modflow()

laytyp= [1,3,0]
layavg= [0,0,0]
chani= [1,1,1]
layvka= [1,1,1]
laywet= [0,0,0]
ipakcb= 1
hdry=-1e+30
iphdry=0
hk=1.0
hani=1.0
vka=1.0
ss=1e-05
sy=0.15
vkcb=0.0
noparcheck=False
extension='upw'
unitnumber=None
filenames=None

In [ ]:
flopy.modflow.ModflowUpw(mnew,
                        model,
    laytyp=laytyp,
    layavg=layavg,
    chani=1.0,
    layvka=0,
    laywet=0,
    ipakcb=None,
    hdry=-1e+30,
    iphdry=0,
    hk=1.0,
    hani=1.0,
    vka=1.0,
    ss=1e-05,
    sy=0.15,
    vkcb=0.0,
    noparcheck=False,
    extension='upw',
    unitnumber=None,
    filenames=None,)

In [ ]:
m.change_model_ws('RR_2022')

In [ ]:
m.write_input()

In [ ]:

m.plot()

In [ ]:
m.dis.export('GIS\grid.shp')

In [ ]:
os.mkdir('GIS')

In [ ]:
g = gpd.read_file("GIS\grid.shp")

g